In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

import tensorflow_hub as hub

# データの読み込み
root_dir = "/kaggle/"
all_train_images = np.load(root_dir + "input/kuzushiji/k49-train-imgs.npz")["arr_0"]
test_images = np.load(root_dir + "input/kuzushiji/k49-test-imgs.npz")["arr_0"]
all_train_labels = np.load(root_dir + "input/kuzushiji/k49-train-labels.npz")["arr_0"]
test_labels = np.load(root_dir + "input/kuzushiji/k49-test-labels.npz")
class_map_df = pd.read_csv(root_dir + "input/kuzushiji/k49_classmap.csv")

# データの分割
train_images, valid_images, train_labels, valid_labels = train_test_split(all_train_images, all_train_labels,
                                                                          test_size=0.2, random_state=42)

2022-03-24 03:19:57.639264: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-24 03:19:57.639377: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# データセットに関するパラメータ
NUM_CLASSES = 49
ORIGINAL_HEIGHT = 28
ORIGINAL_WIDTH = 28

IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32
IMAGE_CHANNEL = 3

# 学習のパラメータ
NUM_EPOCH = 10
BATCH_SIZE = 256

In [3]:
# 画像の前処理
def image_preprocess(image):
    image = tf.image.resize(tf.reshape(tf.repeat(image, IMAGE_CHANNEL, axis=1),(ORIGINAL_HEIGHT, ORIGINAL_WIDTH, IMAGE_CHANNEL)), (IMAGE_HEIGHT, IMAGE_WIDTH))
    image = tf.image.convert_image_dtype(image, dtype=tf.float32) / 255
    return image


# one-hotエンコーディング
def label_preprocess(label):
    label = tf.one_hot(label, 49)
    return label

# modelの定義
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.CategoricalCrossentropy()
model = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2", trainable=True), tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')])
model.build([None, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNEL])
model.compile(optimizer=optimizer, loss=loss_object, metrics=["accuracy"])

2022-03-24 03:20:05.911588: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-24 03:20:05.914867: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-24 03:20:05.914910: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-24 03:20:05.914937: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (1793952ff325): /proc/driver/nvidia/version does not exist
2022-03-24 03:20:05.915232: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [4]:
# データセットの定義
# 注意： tf.data.Dataset.from_tensor_slicesはnumpyの配列を入れるとグラフ変換後も追跡しメモリを消費してしまうため、Tensorへの変換が必要
train_dataset = tf.data.Dataset.from_tensor_slices((tf.constant(train_images, dtype="uint8"), tf.constant(train_labels, "uint8")))
valid_dataset = tf.data.Dataset.from_tensor_slices((tf.constant(valid_images, dtype="uint8"), tf.constant(valid_labels, "uint8")))
test_dataset = tf.data.Dataset.from_tensor_slices((tf.constant(test_images, dtype="uint8"),))

# modelに渡す用のデータセットの変換
# shuffleでデータセットをエポックごとにシャッフル
# mapを使うことでマルチスレッドでの前処理が可能
# prefetchを使うことでデータセットを一定数事前に準備可能
train_ds = (train_dataset.shuffle(len(train_dataset), reshuffle_each_iteration=True).map(lambda image,label: (image_preprocess(image), label_preprocess(label)), num_parallel_calls = tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

valid_ds = (valid_dataset.map(lambda image, label: (image_preprocess(image), label_preprocess(label)), num_parallel_calls = tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

test_ds = (test_dataset.map(image_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

In [5]:
# modelの学習
history = model.fit(train_ds, validation_data=valid_ds, epochs=NUM_EPOCH)

# modelの予測
valid_pred = np.argmax(model.predict(valid_ds), axis=1)
test_pred = np.argmax(model.predict(test_ds), axis=1)


def balanced_accuracy(y_true, y_pred):
    accs=[]
    for cls in range(49):
        mask = (y_true == cls) 
        cls_acc = (y_pred == cls)[mask].mean()
        accs.append(cls_acc)
    accs = np.mean(accs)
    return accs

print(balanced_accuracy(valid_labels, valid_pred))
print(balanced_accuracy(test_labels, test_pred))

2022-03-24 03:20:13.361317: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-24 03:20:13.365982: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


Epoch 1/10
727/727 [==============================] - 672s 909ms/step - loss: 1.9023 - accuracy: 0.5995 - val_loss: 0.2230 - val_accuracy: 0.9372
Epoch 2/10
727/727 [==============================] - 657s 904ms/step - loss: 0.2570 - accuracy: 0.9307 - val_loss: 0.1623 - val_accuracy: 0.9552
Epoch 3/10
727/727 [==============================] - 662s 910ms/step - loss: 0.1850 - accuracy: 0.9507 - val_loss: 0.1455 - val_accuracy: 0.9618
Epoch 4/10
727/727 [==============================] - 659s 906ms/step - loss: 0.1411 - accuracy: 0.9605 - val_loss: 0.1264 - val_accuracy: 0.9649
Epoch 5/10
727/727 [==============================] - 656s 901ms/step - loss: 0.1284 - accuracy: 0.9641 - val_loss: 0.1162 - val_accuracy: 0.9689
Epoch 6/10
727/727 [==============================] - 659s 906ms/step - loss: 0.1030 - accuracy: 0.9706 - val_loss: 0.1248 - val_accuracy: 0.9672
Epoch 7/10
727/727 [==============================] - 652s 896ms/step - loss: 0.0985 - accuracy: 0.9715 - val_loss: 0.2387 -

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice.
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
